<a href="https://colab.research.google.com/github/chrispi21/python-dataeng/blob/main/04_obsluga_plikow_i_manager_kontekstu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Przygotowanie do zajęć

Zacznijmy od pobrania plików z danymi:

In [3]:
!wget -O pracownicy.csv https://raw.githubusercontent.com/chrispi21/python-dataeng/refs/heads/main/pracownicy.csv

--2025-03-01 10:29:28--  https://raw.githubusercontent.com/chrispi21/python-dataeng/refs/heads/main/pracownicy.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1526 (1.5K) [text/plain]
Saving to: ‘pracownicy.csv’

pracownicy.csv      100%[===================>]   1.49K  --.-KB/s    in 0s      

2025-03-01 10:29:28 (20.4 MB/s) - ‘pracownicy.csv’ saved [1526/1526]



Sprawdźmy, czy udało się pobieranie naszego pliku i gdzie się znajduje:

In [7]:
!ls -la pracownicy.csv

-rw-r--r-- 1 root root 1526 Mar  1 10:29 pracownicy.csv


In [5]:
!pwd

/content


# Standardowe podejście

Otwórzmy plik:

In [9]:
sciezka = "/content/pracownicy.csv"
plik = open(sciezka)

Odczytajmy dane:

In [10]:
dane = [
    linia
    for linia in plik.readlines()
]

Dane są w postaci niesparsowanej:

In [11]:
dane

['ID,Imię,Nazwisko,Stanowisko,Departament,Staż pracy (lata)\n',
 '1,Piotr,Kowalski,Inżynier produkcji,Marketing,21\n',
 '2,Ewa,Wiśniewska,Kierownik projektu,Finanse,23\n',
 '3,Jan,Kowalski,Kierownik projektu,HR,5\n',
 '4,Piotr,Lewandowski,Analityk finansowy,Marketing,12\n',
 '5,Piotr,Kamiński,Analityk finansowy,Marketing,23\n',
 '6,Adam,Kowalski,Kierownik projektu,Produkcja,29\n',
 '7,Adam,Szymański,Analityk finansowy,Zarządzanie,19\n',
 '8,Adam,Nowak,Marketing Specialist,HR,15\n',
 '9,Anna,Lewandowski,Inżynier produkcji,Zarządzanie,7\n',
 '10,Maria,Woźniak,Kierownik projektu,HR,30\n',
 '11,Maria,Szymańska,HR Manager,Marketing,12\n',
 '12,Tomasz,Wójcik,Inżynier produkcji,Marketing,29\n',
 '13,Adam,Dąbrowski,Marketing Specialist,HR,14\n',
 '14,Marek,Zieliński,Specjalista ds. IT,IT,13\n',
 '15,Agnieszka,Zielińska,Marketing Specialist,Marketing,25\n',
 '16,Marek,Zieliński,Marketing Specialist,HR,3\n',
 '17,Katarzyna,Nowak,Marketing Specialist,HR,20\n',
 '18,Piotr,Szymański,Inżynier produk

Zamknijmy plik:

In [12]:
plik.close()

In [13]:
# TODO: zapis danych

Wady powyższego podejścia:
1. Konieczność parsowania danych
2. Trzeba zamknąć plik

# Manager kontekstu

Rozwiążemy problem pamiętania o zamknięciu pliku.

In [16]:
sciezka = "/content/pracownicy.csv"
with open(sciezka) as plik:
  dane_v2 = [
      linia
      for linia in plik.readlines()
  ]


Możemy wyświetlić dane:

In [17]:
dane_v2

['ID,Imię,Nazwisko,Stanowisko,Departament,Staż pracy (lata)\n',
 '1,Piotr,Kowalski,Inżynier produkcji,Marketing,21\n',
 '2,Ewa,Wiśniewska,Kierownik projektu,Finanse,23\n',
 '3,Jan,Kowalski,Kierownik projektu,HR,5\n',
 '4,Piotr,Lewandowski,Analityk finansowy,Marketing,12\n',
 '5,Piotr,Kamiński,Analityk finansowy,Marketing,23\n',
 '6,Adam,Kowalski,Kierownik projektu,Produkcja,29\n',
 '7,Adam,Szymański,Analityk finansowy,Zarządzanie,19\n',
 '8,Adam,Nowak,Marketing Specialist,HR,15\n',
 '9,Anna,Lewandowski,Inżynier produkcji,Zarządzanie,7\n',
 '10,Maria,Woźniak,Kierownik projektu,HR,30\n',
 '11,Maria,Szymańska,HR Manager,Marketing,12\n',
 '12,Tomasz,Wójcik,Inżynier produkcji,Marketing,29\n',
 '13,Adam,Dąbrowski,Marketing Specialist,HR,14\n',
 '14,Marek,Zieliński,Specjalista ds. IT,IT,13\n',
 '15,Agnieszka,Zielińska,Marketing Specialist,Marketing,25\n',
 '16,Marek,Zieliński,Marketing Specialist,HR,3\n',
 '17,Katarzyna,Nowak,Marketing Specialist,HR,20\n',
 '18,Piotr,Szymański,Inżynier produk

Nie możemy już odczytać zamkniętego pliku:

In [19]:
plik.readlines()

ValueError: I/O operation on closed file.

Inne zastosowania:
1. Obsługa połączeń (np. do baz danych)
2. Obsługa transakcji w bazach danych
3. Obsługa plików temporalnych i innych zasobów tymczasowych
4. Obsługa innych zasobów, dla których wymagane jest obsłużenie zamknięcia zasobu

Do tworzenie własnych managerów kontekstu można wykorzystać bibiotekę `contextlib` ([link](https://docs.python.org/3/library/contextlib.html)). Posiada ona też wiele gotowych managerów kontekstu.

# Odczyt *csv*

Rozwiążemy teraz problem samodzielnego parsowania zawartości pliku *csv*

In [20]:
import csv

In [33]:
sciezka = "/content/pracownicy.csv"
with open(sciezka,  newline='') as plik_csv:
  dane_csv = list(csv.reader(plik_csv, delimiter=","))

In [34]:
dane_csv

[['ID', 'Imię', 'Nazwisko', 'Stanowisko', 'Departament', 'Staż pracy (lata)'],
 ['1', 'Piotr', 'Kowalski', 'Inżynier produkcji', 'Marketing', '21'],
 ['2', 'Ewa', 'Wiśniewska', 'Kierownik projektu', 'Finanse', '23'],
 ['3', 'Jan', 'Kowalski', 'Kierownik projektu', 'HR', '5'],
 ['4', 'Piotr', 'Lewandowski', 'Analityk finansowy', 'Marketing', '12'],
 ['5', 'Piotr', 'Kamiński', 'Analityk finansowy', 'Marketing', '23'],
 ['6', 'Adam', 'Kowalski', 'Kierownik projektu', 'Produkcja', '29'],
 ['7', 'Adam', 'Szymański', 'Analityk finansowy', 'Zarządzanie', '19'],
 ['8', 'Adam', 'Nowak', 'Marketing Specialist', 'HR', '15'],
 ['9', 'Anna', 'Lewandowski', 'Inżynier produkcji', 'Zarządzanie', '7'],
 ['10', 'Maria', 'Woźniak', 'Kierownik projektu', 'HR', '30'],
 ['11', 'Maria', 'Szymańska', 'HR Manager', 'Marketing', '12'],
 ['12', 'Tomasz', 'Wójcik', 'Inżynier produkcji', 'Marketing', '29'],
 ['13', 'Adam', 'Dąbrowski', 'Marketing Specialist', 'HR', '14'],
 ['14', 'Marek', 'Zieliński', 'Specjalista